### Install the model:
`! pip install ollama llama-index-llms-ollama`
`! sudo snap install ollama`
`! ollama run llama3`

### After downloading the base model and creating the modelfile, we create the parametrised model for our task:
`! ollama create parametrisedEmotionsModel -f Llama3_parameters`

In [4]:
from llama_index.llms.ollama import Ollama
from lib.dataset_utils import load_twitter_data_cleaned, load_goemotions_cleaned
import tqdm
import re
from torch.utils.data import DataLoader

llama3 = Ollama(model="parametrisedEmotionsModel", request_timeout=1000.0)


### Loading Twitter

In [37]:
twitter_train, _, twitter_test = load_twitter_data_cleaned() 
twitter_emotions = """'joy', 'sadness','anger', 'fear', 'love', 'surprise'"""

### Loading Goemotions

In [5]:
import json
label_mapping_path = "./dataset/GoEmotionsSplit/label_mapping.json"

goemotions_train, _, goemotions_test = load_goemotions_cleaned()
json1_file = open(label_mapping_path)
json1_str = json1_file.read()
json1_data = json.loads(json1_str)
goemotions_emotions = json1_data.values()

In [ ]:
TWITTER_BASE_PROMPT = """<|start_header_id|>system<|end_header_id|> Classify the sentences. Choose emotions ONLY among the following: """ + twitter_emotions 
""" 
1. i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived
{
    1: 'joy'
}
2. im updating my blog because i feel shitty
{
    2: 'sadness'
}
3. i can t stop the anxiety i feel when i m alone when i ve got no distractions
4. im trying to feel out my house style now that im living on my own and have creative carte blanche
<|eot_id|>"""

GOEMOTIONS_SINGLE_BASE_PROMPT = """<|start_header_id|>system<|end_header_id|> Classify the sentences. Choose ONLY ONE EMOTION among the following: """ + goemotions_emotions

GOEMOTIONS_MULTI_BASE_PROMPT = """<|start_header_id|>system<|end_header_id|> Classify the sentences. Choose a maximum of three emotions among the following: """ + goemotions_emotions

SAMPLES_STRING = """Here are some samples:"""

TERMINATOR_STRING = """<|eot_id|>"""

In [50]:
# TODO

def get_emotion_between_asterisks(text):
    return re.findall(r'\*\*(.*?)\*\*', text)

def extract_emotions(questions, answers, n_samples):
    for line in answers.split('\n'): # we have (n_samples * 2) newline chars to skip
        pass

class Llama():
    def __init__(self, name, timeout = 1000.0):
        self.model = Ollama(model=name, request_timeout=timeout)
    
    def classify(self, dataset_type, k, train, test, batch_dim, single_label = True, progress_bar = False):
        # executes classification task on model
        # k : number of shots (examples)
        # emotions : list of emotions to be classified 
        # train : training data to fetch examples from
        # test : test data to classify
        # batch_dim : size of batch per prompt
        # single_label : whether to classify single label or multi-label

        samples = None
        if k > 0:
            samples = train.sample(k)

        test_loader = DataLoader(test, batch_size = batch_dim, shuffle = True)

        base_prompt = self.generate_base_prompt(dataset_type, samples, single_label) # to avoid recreating the prompt from scratch at every batch
        for data in tqdm(test_loader, disable=not progress_bar):
            batch_prompt = self.add_test_data_to_prompt(data, base_prompt)
            self.classify_batch(batch_prompt, data)

        self.evaluate()

    def generate_base_prompt(self, dataset_type, samples, single_label = True):

        # spaghetti code but python 3.8 has no switch-case 
        if dataset_type == "twitter":
            if samples:
                return TWITTER_BASE_PROMPT + SAMPLES_STRING
            return TWITTER_BASE_PROMPT
        
        if dataset_type == "goemotions":
            if single_label:
                if samples:
                    return GOEMOTIONS_SINGLE_BASE_PROMPT + SAMPLES_STRING 
                return GOEMOTIONS_SINGLE_BASE_PROMPT    
            if samples:
                return GOEMOTIONS_MULTI_BASE_PROMPT + SAMPLES_STRING
        return  GOEMOTIONS_MULTI_BASE_PROMPT 
                

    def add_test_data_to_prompt(self, test, prompt, samples):
        # appends test data to base prompt
        for index, row in enumerate(test['text']):
            prompt += (str(index) + '. ' + row + '\n')
        prompt += TERMINATOR_STRING
        return prompt

    def classify_batch(self, prompt, data, samples):
        # samples is needed because we need to skip the samples in the response
        # classify batch of data
        pass

    def evaluate(self, data):
        # evaluate the model
        pass

In [51]:
twitter_prompt = """<|start_header_id|>system<|end_header_id|> Classify the sentences. Choose emotions ONLY among the following: """ + twitter_emotions + """ and return with a JSON structure.
 Here are some samples:
1. i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived
{
    1: 'joy'
}
2. im updating my blog because i feel shitty
{
    2: 'sadness'
}
3. i can t stop the anxiety i feel when i m alone when i ve got no distractions
4. im trying to feel out my house style now that im living on my own and have creative carte blanche
<|eot_id|>"""

for index, row in enumerate(twitter_test['text'][60:90]):
    twitter_prompt += (str(index) + '. ' + row + '\n')


In [52]:
response = llama3.complete(twitter_prompt).text

print(response)

{
1: 'joy',
2:  'sadness',
3:  'fear',
4: 'love',
5:  'surprise',
6:  'sadness',
7:  'stress',
8:  'anxiety',
9:  'sadness',
10: 'neither of these emotions is strongly represented',
11:  'shame',
12:  'confusion',
13:  'compassion',
14:  'patriotism',
15:  'anger',
16:  'joy',
17:  'exhaustion',
18:  'relaxation',
19:  'love',
20:  'pride',
21:  'frustration',
22:  'guilt',
23:  'vulnerability',
24:  'sadness',
25:  'unhappiness',
26:  'joy',
27:  'joy',
28:  'neither of these emotions is strongly represented',
29:  'sadness'
}


In [8]:
print(third_prompt)

0. i stood up to you i finally stood up to you and now i feel like im being punished if i could go back and do it again
1. i feel a little nervous i go to the gym
2. i feel like i could go into any situation and become successful because i ve been competing all my life explained schaub in an interview with the a href http bleacherreport
3. i can t stop the anxiety i feel when i m alone when i ve got no distractions
4. im trying to feel out my house style now that im living on my own and have creative carte blanche
5. i have tried to see what it would be like if i liked one of my girl friends but it has never really worked and i can only ever feel an emotional connection to them because they are my friends
6. i had every intention of doing more gardening this morning while it was still cool but i was just feeling so rotten
7. i have a good feeling about this so i am excited
8. i feel like i am just starting to understand the blessings that come from being submissive to the will of the f